In [6]:
import json
import re
from abc import ABC, abstractmethod

class LegacyReportGenerator:
    def generate_report(self, data: dict) -> str:
        """Returns XML format - old system"""
        xml = "<report>\n"
        for key, value in data.items():
            xml += f"  <{key}>{value}</{key}>\n"
        xml += "</report>"
        return xml
        
class ReportGeneratorInterface(ABC):
    @abstractmethod
    def generate(self, data: dict) -> str:
        pass
        
class LegacyReportAdapter(ReportGeneratorInterface):
    def __init__(self, legacy_generator: LegacyReportGenerator):
        self._legacy_generator = legacy_generator
        
    def generate(self, data: dict) -> str:
        xml = self._legacy_generator.generate_report(data)
        # On le convertit et on le renvoie
        return self._xml_to_json(xml)

    def _xml_to_json(self, xml: str) -> str:
        """Simple XML to JSON conversion"""
        result = {}
        matches = re.findall(r"<(\w+)>([^<]+)</\1>", xml)
        
        for key, value in matches:
            try:
                if "." in value:
                    result[key] = float(value)
                else:
                    result[key] = int(value)
            except ValueError:
                result[key] = value
                
        return json.dumps(result)

# 4. Le Client (Le Dashboard moderne)
class AnalyticsDashboard:
    def display(self, json_data: str):
        data = json.loads(json_data)
        print("=== Analytics Dashboard ===")
        for key, value in data.items():
            print(f"  {key}: {value}")

# --- USAGE PROPRE ---
if __name__ == "__main__":
    # Données
    report_data = {"total_sales": 150000, "orders": 1234, "avg_order": 121.55}
    legacy_system = LegacyReportGenerator()
    adapter = LegacyReportAdapter(legacy_system)
    dashboard = AnalyticsDashboard()
    
    print("Generating Report via Adapter...")
    json_output = adapter.generate(report_data)
    dashboard.display(json_output)

Generating Report via Adapter...
=== Analytics Dashboard ===
  total_sales: 150000
  orders: 1234
  avg_order: 121.55
